```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**


# **FLOW DIAGRAMS**

---

`ADD DIAGRAM HERE!`

Required steps:

1. `ADD REQUIRMENT STEP LIST HERE!`

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]'))
CREATE TABLE [VWSSTAGE].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] VARCHAR(50),
	[DATE_FIRST_DAY] VARCHAR(50),
	[VALUE_TYPE] VARCHAR(50),
	[VALUE_NAME] VARCHAR(50),
	[REPORT_STATUS] VARCHAR(50),
	[VALUE] VARCHAR(50)
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]'))
CREATE TABLE [VWSINTER].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME2,
	[DATE_FIRST_DAY] DATETIME2,
	[VALUE_TYPE] VARCHAR(50),
	[VALUE_NAME] VARCHAR(50),
	[REPORT_STATUS] VARCHAR(50),
	[VALUE] INT
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] (
        [DATE_OF_REPORT],
        [DATE_FIRST_DAY],
        [VALUE_TYPE],
        [VALUE_NAME],
        [REPORT_STATUS],
        [VALUE]
    )
    SELECT
        [DBO].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]), --dd-mm-yyyy
        [DBO].[TRY_CONVERT_TO_DATETIME]([DATE_FIRST_DAY]), --yyyy-mm-dd
        [VALUE_TYPE],
        [VALUE_NAME],
        [REPORT_STATUS],
        [VALUE]
    FROM
        [VWSSTAGE].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>VACCINE ADMINISTERED LAST WEEK NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]'))
CREATE TABLE [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME2,
	[DATE_FIRST_DAY] DATETIME2,
	[VALUE_TYPE] VARCHAR(50),
	[VALUE_NAME] VARCHAR(50),
	[REPORT_STATUS] VARCHAR(50),
	[VALUE] INT
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] 
	ON [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]
    ON [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]
AS
BEGIN
    INSERT INTO [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] (
        [DATE_OF_REPORT],
        [DATE_FIRST_DAY],
        [VALUE_TYPE],
        [VALUE_NAME],
        [REPORT_STATUS],
        [VALUE]
    )
    SELECT
        [DATE_OF_REPORT],
        [DATE_FIRST_DAY],
        [VALUE_TYPE],
        [VALUE_NAME],
        [REPORT_STATUS],
        [VALUE]
    FROM
        [VWSINTER].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL])
END;
GO

### **<span style='color:cadetblue'>VIEWS - LAST_WEEK</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] AS
WITH CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_FIRST_DAY],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_REPORT]) AS [DATE_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_FIRST_DAY]) AS [DATE_START_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX](DATEADD(DAY, 6, [DATE_FIRST_DAY])) AS [DATE_END_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
        [VALUE_NAME] AS [VACCINE_TYPE_NAME],
        [VALUE] AS [VACCINE_TYPE_VALUE]
    FROM [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] WITH(NOLOCK))
)
SELECT
    [DATE_UNIX],
    [DATE_START_UNIX],
    [DATE_END_UNIX],
    [DATE_OF_INSERTION_UNIX],
    [VACCINE_TYPE_NAME],
    [VACCINE_TYPE_VALUE]
FROM CTE
WHERE [DATE_START_UNIX] = (
    SELECT MAX([DATE_START_UNIX]) 
    FROM CTE
    WHERE DATE_FIRST_DAY <  ([dbo].[WEEK_START](DATE_OF_REPORT))
)
AND [VACCINE_TYPE_NAME] <> 'unknown'
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION - LAST_4_WEEKS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINE_ADMINISTERED_LAST_4_WEEKS_NL] AS
WITH CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_FIRST_DAY],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_REPORT]) AS [DATE_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_FIRST_DAY]) AS [DATE_START_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX](DATEADD(DAY, 6, [DATE_FIRST_DAY])) AS [DATE_END_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
        [VALUE_NAME] AS [VACCINE_TYPE_NAME],
        [VALUE] AS [VACCINE_TYPE_VALUE]
    FROM [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL] WITH(NOLOCK))
)
SELECT
    [DATE_UNIX],
    [DATE_START_UNIX],
    [DATE_END_UNIX],
    [DATE_OF_INSERTION_UNIX],
    [VACCINE_TYPE_NAME],
    [VACCINE_TYPE_VALUE]
FROM CTE
WHERE [DATE_START_UNIX] = (
    SELECT MAX([DATE_START_UNIX]) 
    FROM CTE
    WHERE DATE_FIRST_DAY <  ([dbo].[WEEK_START](DATE_OF_REPORT))
)
AND [VACCINE_TYPE_NAME] <> 'unknown'
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION - LAST_WEEK</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered_last_week',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'DATE_UNIX|DATE_END_UNIX|DATE_START_UNIX|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 3,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINE_ADMINISTERED_LAST_WEEK_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered_last_week/vaccine_types',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'VACCINE_TYPE_NAME|VACCINE_TYPE_VALUE',
        @layout_type_id INT = 4,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION - LAST_4_WEEKS</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINE_ADMINISTERED_LAST_4_WEEKS_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered_last_4_weeks',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'DATE_UNIX|DATE_END_UNIX|DATE_START_UNIX|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 3,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINE_ADMINISTERED_LAST_4_WEEKS_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered_last_4_weeks/vaccine_types',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'VACCINE_TYPE_NAME|VACCINE_TYPE_VALUE',
        @layout_type_id INT = 4,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO